# Shared LSTM Model - SciTail Dataset

The following notebook contains the implementation of the shared LSTM model for the SciTail dataset

In [24]:
# Import the necessary libraries

import re
from numpy import array
from numpy import asarray
from numpy import zeros

import tensorflow as tf
import keras
import keras.backend as K
from keras.models import Model
from keras.regularizers import L2
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Dropout, Input, LSTM
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping, ModelCheckpoint

import nltk
nltk.download('punkt')

from gensim import models

[nltk_data] Downloading package punkt to
[nltk_data]     /home/aakashj2412/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Helper function that cleans the input data and enumerates the labels

def extract(s):
    s = re.sub('\\(', '', s)
    s = re.sub('\\)', '', s)
    s = re.sub('\\s{2,}', ' ', s)
    return s.strip()

labels = {'entails\n': 0, 'contradiction': 1, 'neutral\n': 2}

In [3]:
# Function that reads data and parses data from file

def readFileData(filePath):
    with open(filePath, 'r') as f:
        inputRows = [row.split('\t') for row in f.readlines()[1:]]

    inputPremises = [row[0] for row in inputRows if row[2] in labels]
    inputHypotheses = [row[1] for row in inputRows if row[2] in labels]
    inputLabels = [labels[row[2]] for row in inputRows if row[2] in labels]
    f.close()

    return [inputPremises, inputHypotheses, inputLabels]

In [6]:
# Reading train and test data

datasetPath = "../../Datasets/SciTail/"
trainData = readFileData(f'{datasetPath}scitail_1.0_train.tsv')
testData = readFileData(f'{datasetPath}scitail_1.0_test.tsv')
validationData = readFileData(f'{datasetPath}scitail_1.0_dev.tsv')

In [7]:
# Hyperparameters

maxLen = 32
epochs = 1000
batchSize = 512
gloveDimension = 300
hiddenDimension = 100
regularization = 4e-6

In [8]:
# Tokenizer to generate the vocabulary of the system

tokenizer = Tokenizer()
tokenizer.fit_on_texts(trainData[0] + trainData[1])
vocabSize = len(tokenizer.word_index)+1

In [9]:
# Convert the train data to sequences as per the vocabulary
trainData[0] = tokenizer.texts_to_sequences(trainData[0])
trainData[1] = tokenizer.texts_to_sequences(trainData[1])

# Pad or trim all generated sequences to the same max sentence length
trainData[0] = pad_sequences(trainData[0], maxLen, padding='post')
trainData[1] = pad_sequences(trainData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
trainData[2] = tf.keras.utils.to_categorical(trainData[2], num_classes=3)

In [10]:
# Convert the test data to sequences as per the vocabulary
testData[0] = tokenizer.texts_to_sequences(testData[0])
testData[1] = tokenizer.texts_to_sequences(testData[1])

# Pad or trim all generated sequences to the same max sentence length
testData[0] = pad_sequences(testData[0], maxLen, padding='post')
testData[1] = pad_sequences(testData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
testData[2] = tf.keras.utils.to_categorical(testData[2], num_classes=3)

In [11]:
# Convert the validation data to sequences as per the vocabulary
validationData[0] = tokenizer.texts_to_sequences(validationData[0])
validationData[1] = tokenizer.texts_to_sequences(validationData[1])

# Pad or trim all generated sequences to the same max sentence length
validationData[0] = pad_sequences(validationData[0], maxLen, padding='post')
validationData[1] = pad_sequences(validationData[1], maxLen, padding='post')

# Transform the labels to one-hot encoding
validationData[2] = tf.keras.utils.to_categorical(validationData[2], num_classes=3)

In [12]:
# Generate the embeddings dictionary from Google News word2vec

word2vecpath = "../../Datasets/Word2Vec/"
embeddingsDict = models.KeyedVectors.load_word2vec_format(f'{word2vecpath}GoogleNews-vectors-negative300.bin', binary=True)

In [ ]:
# Alternative GloVe embeddings

'''
embeddingsDict = dict()
glove = open(r'/content/drive/MyDrive/glove.840B.300d.txt', encoding='utf8')

for line in glove:
    records = line.split()
    word = ''.join(records[:-300])
    vectorDimensions = asarray(records[-300:], dtype='float32')
    embeddingsDict[word] = vectorDimensions

glove.close()
'''

In [14]:
# Iterate through the embeddings and store only those that are present in our vocabulary
'''
embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    vec = embeddingsDict.get(word)
    if vec is not None:
        embeddingsMat[index] = vec
'''

embeddingsMat = zeros((vocabSize, gloveDimension))
for word, index in tokenizer.word_index.items():
    if index % 2500 == 0:
        print(index)
    if word in embeddingsDict.index_to_key:
        embeddingsMat[index] = embeddingsDict[word]

2500
5000
7500
10000
12500
15000
17500
20000


In [15]:
# Define the embedding layer for our baseline RNN model
embed = Embedding(vocabSize, gloveDimension, weights=[embeddingsMat], input_length=maxLen, trainable=False)

# As Premise and Hypothesis are distinct and are to be inputted separately, define two inputs and embed
premise = Input(shape=(maxLen,), dtype='int32')
hypothesis = Input(shape=(maxLen,), dtype='int32')

premInput = embed(premise)
hypoInput = embed(hypothesis)

convert = Dense(hiddenDimension, activation='tanh', input_shape=(gloveDimension,))

premInput = convert(premInput)
hypoInput = convert(hypoInput)

In [16]:
# Once the sentence embeddings have been generated, generate a matrix of dimensions maxLen X gloveDimension
# On adding maxLen, we get a single embedding vector of length gloveDimension

#rnn = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=hiddenDimension)
#rnn = LSTM(hiddenDimension, dropout=0.2, recurrent_dropout=0.2)

rnnH = LSTM(hiddenDimension, dropout=0.2, return_state=True)
rnnP = LSTM(hiddenDimension, dropout=0.2)

In [17]:
# Apply batch normalization to the two input embeddings separately

premInput, sH, sC = rnnH(premInput)
hypoInput = rnnP(hypoInput, initial_state=[sH, sC])
premInput = tf.keras.layers.BatchNormalization()(premInput)
result = tf.keras.layers.BatchNormalization()(hypoInput)

In [18]:
# Joint is a concatenated embeddings layer, generated from the premise and hypothesis inputs
# Dilution of probability 0.2, to assist in regularization
#joint = keras.layers.concatenate([premInput, hypoInput])
result = Dropout(0.2)(result)
'''
for i in range(3):
    result = Dense(2*hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
    result = Dropout(0.2)(result)
    result = tf.keras.layers.BatchNormalization()(result)
'''

# 3 layers of the TanH activation function, along with L2 regularization.
# The final decision is based on the Softmax function

result = Dense(hiddenDimension, activation='tanh', kernel_regularizer=L2(regularization))(result)
pred = Dense(3, activation='softmax')(result)

In [19]:
# Defining the final models input and output format, as well as compilation parameters

model = Model(inputs=[premise, hypothesis], outputs=pred)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
# Fitting the model using the train data

callback = EarlyStopping(monitor='val_loss', min_delta=0, patience=50)
mcCallback = ModelCheckpoint('./2LSTM_sci.h5', monitor='val_loss')
model.fit([array(trainData[0]), array(trainData[1])], array(trainData[2]), batch_size=batchSize, epochs=epochs, callbacks=[callback, mcCallback], validation_data=[[array(validationData[0]), array(validationData[1])], array(validationData[2])])

Epoch 1/1000
47/47 [==============================] - 16s 331ms/step - loss: 0.1744 - accuracy: 0.9322 - val_loss: 1.0275 - val_accuracy: 0.6930
Epoch 2/1000
47/47 [==============================] - 17s 368ms/step - loss: 0.1710 - accuracy: 0.9324 - val_loss: 0.9881 - val_accuracy: 0.6470
Epoch 3/1000
47/47 [==============================] - 16s 349ms/step - loss: 0.1669 - accuracy: 0.9341 - val_loss: 1.0195 - val_accuracy: 0.6616
Epoch 4/1000
47/47 [==============================] - 17s 369ms/step - loss: 0.1596 - accuracy: 0.9360 - val_loss: 0.7334 - val_accuracy: 0.6807
Epoch 5/1000
47/47 [==============================] - 17s 370ms/step - loss: 0.1570 - accuracy: 0.9385 - val_loss: 1.2739 - val_accuracy: 0.6447
Epoch 6/1000
47/47 [==============================] - 17s 352ms/step - loss: 0.1540 - accuracy: 0.9408 - val_loss: 0.9913 - val_accuracy: 0.6884
Epoch 7/1000
47/47 [==============================] - 17s 352ms/step - loss: 0.1509 - accuracy: 0.9401 - val_loss: 1.2282 - val_ac

In [33]:
model.save('../../Models/2LSTM_sci.h5')

In [37]:
model = keras.models.load_model('../../Models/2LSTM_sci.h5')

In [35]:
# Evaluating accuracy on the trained model

loss, acc = model.evaluate([array(testData[0]), array(testData[1])], array(testData[2]), batch_size=256)
print('Loss = ', loss)
print('Acc = ', acc)

9/9 [==============================] - 1s 65ms/step - loss: 1.6429 - accuracy: 0.6965
Loss =  1.6428693532943726
Acc =  0.6964705586433411


We can thus observe the following results for the shared LSTM model, for the SciTail dataset:

- Training Accuracy: 97.97%
- Validation Accuracy: 70.53%
- Test Accuracy: 69.64%

We can see, based on the training and test data, that the model has slightly overfit on the data. This can be controlled to a certain degree by optimizing the patience value for early stopping and other hyperparameters. However, as we observe in the SICK dataset, the amount of data of the dataset isn't sufficient for the shared LSTM model, and it is thus suitable for larger datasets like SNLI.